From the guide, and our dataset, we will need (in no particular order):

1) time_series_data = a dictionary 
    {
    site_id : 
        {
        date_time : rs_annual,
        date_time : rs_annual
        },
    site_id : 
        {
        date_time : rs_annual,
        date_time : rs_annual
        }
    }

2) geo_df = is a geopandas DataFrame of Lat & Long, index = site_id

then we can display our geo_df in folium, using style_dict for the choropleth slider

#objectives#

We want to show a series of data sites that have collected readings over time. Using a slider we should be able to visualise those changes over time.

The core data set is from: https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1827
...a near-universal compendium of published soil respiration (Rs) data. The database encompasses published studies...

We are mainly concerned with each study_id, it's lat & long, study_midyear, and rs_annual (a measure of soil CO2 respiration).

our guide
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb

In [1]:
import pandas as pd

df_all_data = pd.read_csv('srdb-data-V5.csv')[['Latitude', 'Longitude', 'Study_midyear', 'Study_number', 'Rs_annual', 'Site_name']].dropna().astype(str)

/home/digiadmin/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,39,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
from datetime import datetime
time_series_data = {}
for study_id in df_all_data['Study_number'].unique():
    time_series = {}
    for study_data in df_all_data[df_all_data['Study_number']==study_id].set_index(['Study_number']).values:
        dt_time = study_data[2][:4] + '-06-01'
        dt_obj = datetime.strptime(dt_time, '%Y-%m-%d')
        millisec = dt_obj.timestamp()
        dt_index = str(millisec)
        time_series[dt_index] = study_data[3]
    time_series_data[study_id] = time_series
#print(time_series_data)

In [3]:
# our geojson dictionary
geo_data = { "type": "FeatureCollection", "features": [] }
for study_id in df_all_data['Study_number'].unique():
    # each site / feature in our data set
    site_data = {
                "type" : "Feature", 
                "geometry" : {
                    "type" : "Point",
                    },
                "properties": {}
                }
    for study_data in df_all_data[df_all_data['Study_number']==study_id].set_index(['Study_number']).values:
        site_data["geometry"]["coordinates"] = [study_data[1], study_data[0]]
        site_data["properties"]["name"] = study_data[4]
    geo_data["features"].append(site_data)
#print(geo_data)

In [4]:
import folium
from folium.plugins import TimeSliderChoropleth

m = folium.Map([0, 0], tiles="Stamen Toner", zoom_start=2)

g = TimeSliderChoropleth(
    geo_data,
    styledict=time_series_data,
).add_to(m)
# time slider not chronological
# data points don't appear to show different data values
# crashes my laptop - could be memory issue
m